# POC - Entity Resolution and Data Quality Analysis 

## Scope Statement
This notebook aims to document a Proof-of-Concept simulation for Veridion Company.
The goal is to demonstrate the value of entity resolution in creating a single and accurate view of data in order to improve data quality, compliance and analytics.

## Task description
A client provided a sample of supplier companies for the POC. The sample was processed through the company's entity resolution engine, which returned up to five candidate matches for each original company. 

The role of the Data Analyst is to analyze and select the best match for each provided company, returning a single, clean, accurate, and comprehensive view of the data. Furthermore, the Data Analyst has the important responsability of demonstrating to the client a professional approach to working with data and supporting business decisions.

### Approach 
The approach for this POC is to use analytical judgement on the results of the entity resolution engine in order to deliver a clean and reliable supplier dataset. The main work related to matching patterns is already handled by the entity resolution engine, so the Data Analyst’s responsibility is to interpret the results and select the most accurate match.

The first step is to review the candidate matches using domain specific attributes for each company and select the one that best represents the real world company.

The next step is to analyze companies that don't appear to have an accurate match and to identify the main factors that could lead to a better match. This includes consulting the company website, searching online, identifying patterns related to the original company, and, as a last resort, leaving the record unmatched due to insufficient company details. 

Last step is to provide a clean and accurate dataset containing the selected matches for the original companies, allowing the client to easily view and understand the companies' attributes and details for future improvements in procurement activities.

## Starting by loading and analyzing the data

In [1]:
import pandas as pd

df = pd.read_csv("presales_data_sample.csv")
df.head(10)

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,...,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at
0,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,26e22210-93e5-11eb-b997-8dd98d09cf25,...,NaN,NaN,http://www.linkedin.com/company/mnet-services-...,NaN,NaN,NaN,NaN,web servers: apache http server - 2 | javascri...,2020-02-25T14:47:51.000Z,2024-11-29T04:18:00.109Z
1,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,01004641-1dd8-11ef-9268-316fc8e174dd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-29T16:24:11.019Z,2025-04-20T15:03:24.026Z
2,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,8266efc1-13e7-11ec-aa14-7bf90e1e10f1,...,https://twitter.com/Network24seven%20,NaN,https://www.linkedin.com/company/51633612%20%20,NaN,NaN,NaN,NaN,NaN,2021-09-12T05:28:48.000Z,2025-03-18T23:08:37.059Z
3,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,0183f0b2-93e5-11eb-be5a-4f810ab55f2e,...,https://twitter.com/emeriosoft,NaN,https://www.linkedin.com/company/emeriosoft,NaN,NaN,NaN,NaN,miscellaneous: popper | maps: google maps | pr...,2020-05-03T12:33:22.000Z,2025-03-31T16:16:58.462Z
4,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,87bb7cde-93e4-11eb-8474-3bbe2d07207d,...,https://twitter.com/AsiaticPR,https://www.instagram.com/asiaticpublicrelations/,https://www.linkedin.com/company/asiaticpublic...,NaN,NaN,https://www.youtube.com/channel/UClKgHvHIuOu4K...,NaN,miscellaneous: babel | javascript libraries: c...,2020-02-19T03:58:25.000Z,2024-11-25T11:35:47.963Z
5,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,8b99ef83-93e4-11eb-b997-8dd98d09cf25,...,NaN,NaN,https://www.linkedin.com/company/2operate-as,NaN,NaN,NaN,NaN,javascript libraries: lodash | javascript libr...,2020-02-17T15:56:51.000Z,2025-03-28T15:16:32.101Z
6,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,92d0f2d4-0581-11ec-8c14-3f55c5871ced,...,https://twitter.com/co2operate,NaN,NaN,NaN,NaN,NaN,NaN,miscellaneous: rss | cms: wordpress - 5.9.7 | ...,2021-08-22T23:51:37.000Z,2024-11-05T09:16:37.000Z
7,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,06393b6c-93e5-11eb-86dd-ffbd1b53140e,...,NaN,NaN,https://www.linkedin.com/company/1508857,NaN,NaN,NaN,NaN,cms: wordpress - 6.3.2 | javascript libraries:...,2020-08-21T08:47:11.000Z,2025-03-31T14:34:16.681Z
8,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,0b470651-93e5-11eb-b4a6-27899be4e45d,...,https://twitter.com/2APharma,NaN,https://www.linkedin.com/company/2a-pharma-ab,NaN,NaN,NaN,NaN,widgets: instagram feed for wordpress | widget...,2020-02-17T15:49:48.000Z,2025-02-17T10:36:39.000Z
9,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,01db9ab2-06b1-11ec-984c-7f34b099c1c2,...,NaN,NaN,https://www.linkedin.com/company/rensams-solut...,NaN,NaN,NaN,NaN,web servers: empathy platform | widgets: css |...,2021-08-26T10:07:38.000Z,2025-03-28T00:38:27.256Z


In [2]:
print(df.isnull().sum())
df.info()

input_row_key                 0
input_company_name            0
input_main_country_code       0
input_main_country            0
input_main_region           335
                           ... 
youtube_url                2469
tiktok_url                 2951
technologies               1191
created_at                    0
last_updated_at               0
Length: 76, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951 entries, 0 to 2950
Data columns (total 76 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   input_row_key                 2951 non-null   int64  
 1   input_company_name            2951 non-null   object 
 2   input_main_country_code       2951 non-null   object 
 3   input_main_country            2951 non-null   object 
 4   input_main_region             2616 non-null   object 
 5   input_main_city               2566 non-null   object 
 6   input_main_postcode           2409 non-

In [3]:
pct_null_values = (df.isnull().mean() * 100).round(2)
pct_null_values = pct_null_values[pct_null_values > 0].sort_values()
no_null_cols = [col for col in df.columns if col not in pct_null_values]

print(f"Percentage of null values in the dataset for each column: {pct_null_values}%")
print(f"\nTotal number of columns with missing values: {pct_null_values.count()}")
print(f"\nTotal number of columns with more than 50% of missing values: {pct_null_values[pct_null_values > 50].count()}")
print(f"\nColumns with no missing values: {no_null_cols}")
print(f"\nPercentage of missing values for main_country, main_region, main_city provided by the engine:\n{pct_null_values[['main_country', 'main_region', 'main_city']]}")

Percentage of null values in the dataset for each column: generated_business_tags          4.27
generated_description            4.88
num_locations                    4.88
main_country                     4.88
locations                        4.88
                                ...  
naics_2022_secondary_codes      92.71
naics_2022_secondary_labels     92.71
android_app_url                 96.51
ios_app_url                     96.75
tiktok_url                     100.00
Length: 68, dtype: float64%

Total number of columns with missing values: 68

Total number of columns with more than 50% of missing values: 15

Columns with no missing values: ['input_row_key', 'input_company_name', 'input_main_country_code', 'input_main_country', 'veridion_id', 'company_name', 'created_at', 'last_updated_at']

Percentage of missing values for main_country, main_region, main_city provided by the engine:
main_country     4.88
main_region      8.78
main_city       11.18
dtype: float64


## Initial Data Quality Observations

A significant number of columns with many missing values were identified during the initial data loading and inspection phase of our dataset.

Out of a total of 76 columns provided in the data frame, 68 columns contain missing values.

While some columns have a relatively low percentage of missing values, others (15 columns) contain more than 50% null entries.
For the purpose of this POC, it is a good practice to consider columns with more than 50% missing values as a low-confidence attributes and to not include them in our further analysis. These fields are unlikely to have a reliable contribution to entity resolution or to procurement activities. 

The focus should therefore be placed on attributes with higher data confidence, which are more helpful for identifying and validating real world companies.

Also, it is a good practice to check the percentage of missing values for key engine-provided location attributes (such as country, region, and city), as they play an important role in evaluating candidate matches.

In [4]:
high_null_cols = pct_null_values[pct_null_values > 50]

high_null_cols = high_null_cols.index.tolist()
high_null_cols

['linkedin_url',
 'year_founded',
 'ibc_insurance_codes',
 'ibc_insurance_labels',
 'facebook_url',
 'emails',
 'instagram_url',
 'twitter_url',
 'youtube_url',
 'other_emails',
 'naics_2022_secondary_codes',
 'naics_2022_secondary_labels',
 'android_app_url',
 'ios_app_url',
 'tiktok_url']

## Identifying Key Attributes

The key attributes are based mainly on those processed by the entity resolution engine, such as company name, country, city, website, industry and sector, as well as the attributes provided by the client (company name, country, city, and postcode and main street where no null values are present). The entity resolution engine offers the key data, the client's input offers the context, the main task is to validate those together.

As a first step, text standardization should be performed on the data to eliminate the risk of mismatched values caused by data inconsistencies.

As a second step, a comparison between the details listed above should be performed. After eliminating the entries that are not a match across these keys attributes, the analysis can focus on additional relevant information, such as the website of the company, the generated company description, business tag, the industry and sector attributes, in order to indentify meaningful relationships between those attributes and confirm the most accurate match.

It is important to understand that keys attributes alone are not enough to support the analysis when identifying common characteristics, but when these attributes are evaluated in combination with other relevant attributes, they can make a significant difference.

In [52]:
# transform in lowercase and remove spaces
# cols = df.select_dtypes(include="object").columns.tolist()
# for col in cols:
#     df[col] =(
#         df[col]
#         .astype("string")
#         .str.lower()
#         .str.strip()
#     )
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951 entries, 0 to 2950
Data columns (total 76 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   input_row_key                 2951 non-null   int64  
 1   input_company_name            0 non-null      float64
 2   input_main_country_code       0 non-null      float64
 3   input_main_country            0 non-null      float64
 4   input_main_region             0 non-null      float64
 5   input_main_city               0 non-null      float64
 6   input_main_postcode           0 non-null      float64
 7   input_main_street             0 non-null      float64
 8   input_main_street_number      0 non-null      float64
 9   veridion_id                   0 non-null      float64
 10  company_name                  0 non-null      float64
 11  company_legal_names           0 non-null      float64
 12  company_commercial_names      0 non-null      float64
 13  mai

### Bug Identified
Pandas transformed all object type columns into float types after several attempts to convert string values to lowercase.

As a result, an alternative approach is required for standardizing text values. For the moment, for lowercase and removed spaces, the focus can be applied only on the key attributes relevant for comparison, rather than the entire dataset.

From the client-provided data, the key attributes are:
- input_company_name
- input_main_country
- input_main_region
- input_main_city

Attributes provided by the client starts with an "input" keyword. 

From the entity resolution engine, the key attributes are:
- company_name
- main_country
- main_region
- main_city
- long_description
- main_industry
- website_domain
- website_url

In [5]:
df_work = df.copy()

text_cols = [
    "input_company_name", 
    "input_main_country", 
    "input_main_region", 
    "input_main_city", 
    "company_name",
    "main_country",
    "main_region",
    "main_city",
    "long_description",
    "main_industry",
    "website_domain",
    "website_url"
]
df_work = df_work[text_cols]
df_work.head(10)

,input_company_name,input_main_country,input_main_region,input_main_city,company_name,main_country,main_region,main_city,long_description,main_industry,website_domain,website_url
0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,Pakistan,Sindh,Karachi,New Millennium Network,Pakistan,Sindh,Karachi,"Incorporated in 2001, MNET is the first compan...",Telecommunication Services,mnet.com.pk,https://www.mnet.com.pk/
1,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,Pakistan,Sindh,Karachi,Private Helipad Ali Villa,Pakistan,Sindh,Karachi,NaN,NaN,NaN,NaN
2,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,Pakistan,Sindh,Karachi,24seven Research Network,India,Haryana,Faridabad,"With a team of highly skilled, qualified and m...",Marketing & Advertising Agencies,24sevenresearchnetwork.com,https://24sevenresearchnetwork.com/
3,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,Pakistan,Sindh,Karachi,EmerioSoft,Pakistan,Sindh,NaN,Brand Identity 'We are a Digital Agency that o...,Digital Agencies,emeriosoft.com,https://www.emeriosoft.com/
4,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,Pakistan,Sindh,Karachi,Asiatic Public Relations Network,Pakistan,Sindh,Karachi Division,Asiatic Public Relations Network (Private) Lim...,Marketing & Advertising Agencies,apr.com.pk,https://apr.com.pk/
5,2OPERATE A/S,Denmark,North Denmark Region,Aalborg,2operate,Denmark,North Denmark Region,Aalborg Municipality,"2operate deliver high-quality software, servic...",Telecommunication Services,2operate.com,https://www.2operate.com/
6,2OPERATE A/S,Denmark,North Denmark Region,Aalborg,CO 2 Operate,NaN,NaN,NaN,Samen bouwen aan de transitie naar een duurzam...,Engineering Services,co2operate.com,https://co2operate.com/
7,2OPERATE A/S,Denmark,North Denmark Region,Aalborg,Conzentrate,Denmark,Capital Region Of Denmark,Herlev,Learning Arena LMS er et digitalt læringsunive...,Professional Schools,conzentrate.com,https://conzentrate.com/
8,2OPERATE A/S,Denmark,North Denmark Region,Aalborg,2A Pharma ApS.,Denmark,North Denmark Region,Aalborg,Previous Next 2A Pharma is a clinical stage Sw...,Pharmaceuticals,2apharma.com,https://2apharma.com/
9,2OPERATE A/S,Denmark,North Denmark Region,Aalborg,RenSams Solutions,Denmark,North Denmark Region,Aalborg,"T RenSams Solutions IVS,we sell and purchase n...",Medical Equipment & Supplies,rensams.dk,https://rensams.dk/


In [6]:
# transform lowercase and remove spaces
df_work = (
    df_work.apply(
        lambda col: col.map(lambda x: x.lower().strip() if isinstance(x, str) else x)
    )
)
df_work

# process each client-provided company through the entity resolution steps
list_of_company = df_work["input_company_name"].drop_duplicates().tolist()
print(f"Number of unique companies provided by client: {len(list_of_company)}")

Number of unique companies provided by client: 589


After reducing noise caused by text inconsistencies, the total number of unique companies provided by the client is 589. 
It is a good practice to keep track of this number throughout the entity resolution workflow in order to avoid the risk of eliminting any of the unique companies.

### Matching Columns

The analysis can now focus on matching the cleaned and relevant columns. 
The matching process will follow these steps:
1. Match the client-provided country with the engine-provided country, as well as region and city where available.
2. Identify common keywords between the client company name and the engine company name or website domain.
3. Evaluate potential red flags and keep what best fits the ovarall matching signals.

A red flag can be a missing value in one or more key attributes, a mismatch between columns or inconsistencies information across attributes (like location for example).

In [7]:
df_work["country_match"] = (
    df_work["input_main_country"] == df_work["main_country"]
)
print(df_work["country_match"].value_counts(dropna=False))

df_work["region_match"] = (
    df_work["input_main_region"] == df_work["main_region"]
)
print(f'\n{df_work["region_match"].value_counts(dropna=False)}')

df_work["city_match"] = (
    df_work["input_main_city"] == df_work["main_city"]
)
print(f'\n{df_work["city_match"].value_counts(dropna=False)}')

df_work

country_match
True     1985
False     966
Name: count, dtype: int64

region_match
False    1727
True     1224
Name: count, dtype: int64

city_match
False    2151
True      800
Name: count, dtype: int64


,input_company_name,input_main_country,input_main_region,input_main_city,company_name,main_country,main_region,main_city,long_description,main_industry,website_domain,website_url,country_match,region_match,city_match
0,24-seven media network (private) limited,pakistan,sindh,karachi,new millennium network,pakistan,sindh,karachi,"incorporated in 2001, mnet is the first compan...",telecommunication services,mnet.com.pk,https://www.mnet.com.pk/,True,True,True
1,24-seven media network (private) limited,pakistan,sindh,karachi,private helipad ali villa,pakistan,sindh,karachi,NaN,NaN,NaN,NaN,True,True,True
2,24-seven media network (private) limited,pakistan,sindh,karachi,24seven research network,india,haryana,faridabad,"with a team of highly skilled, qualified and m...",marketing & advertising agencies,24sevenresearchnetwork.com,https://24sevenresearchnetwork.com/,False,False,False
3,24-seven media network (private) limited,pakistan,sindh,karachi,emeriosoft,pakistan,sindh,NaN,brand identity 'we are a digital agency that o...,digital agencies,emeriosoft.com,https://www.emeriosoft.com/,True,True,False
4,24-seven media network (private) limited,pakistan,sindh,karachi,asiatic public relations network,pakistan,sindh,karachi division,asiatic public relations network (private) lim...,marketing & advertising agencies,apr.com.pk,https://apr.com.pk/,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,comba technologies sdn. bhd.,malaysia,selangor,subang jaya,v-smart technologies sdn bhd.,malaysia,selangor,semenyih,we are highly committed in providing user frie...,software & it services,vsmart.com.my,http://vsmart.com.my/v1/,True,True,False
2947,comba technologies sdn. bhd.,malaysia,selangor,subang jaya,mydialysis,malaysia,selangor,cyberjaya,living with flexible & healthy lifestyle work ...,hospitals & clinics,mydialysis.com.my,https://mydialysis.com.my/,True,True,False
2948,comba technologies sdn. bhd.,malaysia,selangor,subang jaya,cm boost studio,malaysia,selangor,subang jaya,"at cm boost studio, we thrive at the heart of ...",marketing & advertising agencies,cmbooststudio.com,https://www.cmbooststudio.com/,True,True,True
2949,comba technologies sdn. bhd.,malaysia,selangor,subang jaya,comba telecom inc.,united states,california,milpitas,"comba telecom, inc., based in milpitas, califo...",telecommunication services,combausa.com,https://combausa.com/,False,False,False


### Analysis on Matching Columns

The three client-provided columns are very important for our comparison, as they represent the main identifying factors of a company, especially from a location perspective.

However, the matching process does not eliminate missing values. The fact that some client-provided columns do not have values for region or city does not automatically mean that a candidate is not a valid match. In these cases, missing values should be treated as neutral signals rather than direct mismatches.

On the other hand, both the client-provided country and the engine-provided country contain no missing values. This makes country matching a stronger and more reliable signal. The presence of 966 false country matches can be considered a significant red flag and should be reviewed during the final matching decision.

Overall, country match is considered the strongest consistency check, while region and city matching provide additional context that helps increase or decrease confidence in the final match.

A next step could be to verify which companies with False value for country match also have True values for both region and city matching. After this check, two interpretations can be confirmed:
1. Records with False country match but with True values for both region and city may indicate a possible missing or inconsistent country value rather than a true mismatch.
2. If country, region, and city are all False, this can be interpreted as a strong red flag and suggests a higher chance of an incorrect match.

In [8]:
country_false = df_work[df_work["country_match"] == False]

region_city_true = country_false[(country_false["region_match"] == True) & (country_false["city_match"] == True)]
print(f"The length of false country with true region and city: {len(region_city_true)}")

all_location_false = country_false[(country_false["region_match"] == False) & (country_false["city_match"] == False)]
print(f"The length of all location false: {len(all_location_false)}")

The length of false country with true region and city: 0
The length of all location false: 966


In [9]:
country_false[[
    "input_main_country", 
    "input_main_region", 
    "input_main_city",
    "main_country",
    "main_region",
    "main_city"
]].head(10)

,input_main_country,input_main_region,input_main_city,main_country,main_region,main_city
2,pakistan,sindh,karachi,india,haryana,faridabad
6,denmark,north denmark region,aalborg,NaN,NaN,NaN
10,norway,viken,bærum,ireland,leinster,dublin 2
11,norway,viken,bærum,slovakia,region of bratislava,bratislava
12,norway,viken,bærum,mauritius,mauritius,quatre bornes
14,norway,viken,bærum,canada,quebec,montreal
15,hong kong,hong kong,hong kong island,united states,california,milpitas
20,malaysia,selangor,subang jaya,italy,piedmont,garbagna novarese
24,malaysia,selangor,subang jaya,united kingdom,wales,flintshire
27,singapore,central,singapore,mexico,tamaulipas,reynosa


### Correct Interpretations
The first hypothesis "country missing, region and city match", is not confirmed. A result of 0 records for this hypothesis and 966 records where country, region, and city are all false confirms the presence of a strong red flag and strongly suggests a higher chance of an incorrect match. 

As a result, these candidates can be removed from the list of possible matches for reducing noise and allowing the analysis to focus on other key attributes that can better support the identification of the correct match. 

An additional group-level check may occur to further refine the candidate set for each client-provided company.

If, within a specific client company group, there is at least one candidate with a confirmed country match and a match on either the region or city attribute, then candidates with no matching for region or city will be excluded from the group, as a stronger alternative already exists. 

But if there is no candidate within the group with a match for one of the region or city field, no candidates will be excluded in order to avoid the risk of elimnating the only possible valid match. 

In [10]:
df_candidates = df_work[df_work["country_match"] == True]
print(f"Shape of data: {df_candidates.shape}")

# process each client-provided company through the entity resolution steps
list_of_company1 = df_candidates["input_company_name"].drop_duplicates().tolist()
print(f"\nNumber of unique companies provided by client: {len(list_of_company1)}\n")

removed_companies = [col for col in list_of_company if col not in list_of_company1]
print(f"Companies excluded after country-level validation due to missing country match (considered unresolved at this stage): {removed_companies}\n")


# create a function for filtering the region and city matches
def filter_by_region_or_city(group):
    has_region_or_city = (
        (group["input_main_region"] == group["main_region"]) |
        (group["input_main_city"] == group["main_city"])
    ).any()

    if has_region_or_city:
        return group[
            (group["region_match"] == True) |
            (group["city_match"] == True)
        ]
    else:
        return group

df_step1 = (
    df_candidates.groupby("input_company_name", group_keys=False)
    .apply(filter_by_region_or_city)
)

# see how the rows are reduced after filtering
print(f"Shape of data: {df_step1.shape}")

# create a more restricted function to include both region and city matches
def filter_by_region_and_city(group):
    has_region_and_city = (
        (group["input_main_region"] == group["main_region"]) &
        (group["input_main_city"] == group["main_city"])
    ).any()

    if has_region_and_city:
        return group[
            (group["region_match"] == True) &
            (group["city_match"] == True)
        ]
    else:
        return group

df_step2 = (
    df_step1.groupby("input_company_name", group_keys=False)
    .apply(filter_by_region_and_city)
)

# see how the rows are reduced after filtering
print(f"Shape of data: {df_step2.shape}")

Shape of data: (1985, 15)

Number of unique companies provided by client: 563

Companies excluded after country-level validation due to missing country match (considered unresolved at this stage): ['egencia sweden ab', 'google cloud emea limited', 'huber+suhner electronics private limited', 'oberthur technologies norway as', 'advis a/s', 'kahoot! denmark aps', 'linkedin ireland unlimited company', 'amdocs development limited', 'nagarro as', 'analysys mason as', 'itelligence a/s', 'anritsu a/s', 'rttd ltd', 'sfdc ireland limited', 'sipm', 'softwareone co., ltd.', 'softwareone pte. ltd.', 'squaretrade europe limited', 'subex (asia pacific) pte. limited', 'subex (uk) limited', 'setec card a/s', 'vertiv (malaysia) sdn. bhd.', 'vertiv (thailand) co., ltd.', 'ziff davis ireland limited', 'aviat networks japan llc', 'a. t. kearney pte. ltd']

Shape of data: (1519, 15)
Shape of data: (1301, 15)


/var/folders/yl/brxr15jx04s0b2mrpchp1l9r0000gn/T/ipykernel_28956/1364107024.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_by_region_or_city)
/var/folders/yl/brxr15jx04s0b2mrpchp1l9r0000gn/T/ipykernel_28956/1364107024.py:52: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_by_region_and_city)


After applying country-level validation, the number of unique client-provided companies was reduced to 563. The 26 companies that were excluded from the data didn't have any candidate entities with a matching country. This indicates that the available candidates for these companies are too weak to be considered reliable for further evaluation at this stage.

The next step is to identify any keywords in the cliend-provided company name that match the engine-provided company name or website domain.

In [11]:
# cleaning the noise
import re

input_name_keywords = (
    df_step2["input_company_name"]
    .apply(lambda row: re.split(r"[-()\.\/,\s]+", row))
)

input_name_keywords = input_name_keywords.apply(
    lambda words: [w.lower().strip() for w in words if w]
)

input_name_keywords

0       [24, seven, media, network, private, limited]
1       [24, seven, media, network, private, limited]
8                                    [2operate, a, s]
9                                    [2operate, a, s]
560                               [3, step, it, a, s]
                            ...                      
2713                               [zte, corporation]
2727                                [zte, norway, as]
2728                                [zte, norway, as]
2729                                [zte, norway, as]
2730                                [zte, norway, as]
Name: input_company_name, Length: 1301, dtype: object

In [12]:
# counting the most frequent words
from collections import Counter

all_words = [word for words in input_name_keywords for word in words]

most_frequent = Counter(all_words)

print(most_frequent.most_common(20))

[('ltd', 227), ('s', 222), ('a', 221), ('as', 185), ('limited', 178), ('pte', 112), ('aps', 87), ('sdn', 82), ('bhd', 82), ('ab', 80), ('co', 78), ('services', 66), ('danmark', 55), ('telenor', 47), ('inc', 46), ('technologies', 42), ('singapore', 41), ('private', 39), ('thailand', 36), ('group', 35)]


### Identifying Legal Suffixes

After cleaning and separating the words for each company name provided by the client, a good practice is to identify words that represents legal entity suffixes specific to a country or a company legal structure. 

The advantage of identifying these terms is that they can be ignored during keyword matching because their presence is more likely to intorduce noise rather than help differentiate between companies.

After validating the existence and meaning of those legal suffixes through external search, the next step is to group them into a list and exclude them from the keyword matching process applied to the engine-provided company names.

In [13]:
legal_suffixes = ['ltd', 'limited', 'sdn', 'bhd', 'pte', 'aps', 'ab', 'co', 'inc', 'private']

#rewrite variable after eliminating legal suffixes
input_name_keywords = input_name_keywords.apply(lambda words: [w for w in words if w not in legal_suffixes])

In [14]:
# identify common keywords
def keyword_match(row):
    keywords = input_name_keywords.loc[row.name]
    
    company_name = row["company_name"] 
    website_domain = row["website_domain"] 

    if pd.isna(company_name):
        company_name = ""

    if pd.isna(website_domain):
        website_domain = ""

    return any(
        kw in company_name or kw in website_domain
        for kw in keywords
    )

df_step2["name_keyword_match"] = df_step2.apply(keyword_match, axis=1)
df_step2["name_keyword_match"].value_counts()

name_keyword_match
True     1028
False     273
Name: count, dtype: int64

### Keywords Matching Interpretation
One observation is that there are many keywords that appear in both compared columns. For a more focused matching process and a clear analysis, rows where no keyword exists can be excluded from further consideration.

As a next step, a check to determine how many keywords appear only once and how many appear at least twice can be performed to see if a stricter keyword evaluation is a good practice cand an increase confidence in the match by prioritizing stonger and more consistent keyword signals.

In [15]:
def filter_by_keyword_existence(group):
    has_keyword = group["name_keyword_match"].any()

    if has_keyword:
        return group[group["name_keyword_match"] == True]

    else:
        return group

df_step3 = (
    df_step2.groupby("input_company_name", group_keys=False)
    .apply(filter_by_keyword_existence)
)

input_name_keywords = input_name_keywords.loc[df_step3.index]

# identify keywords count
def count_keywords(row):
    keywords = input_name_keywords.loc[row.name]
    
    company_name = row["company_name"] 
    website_domain = row["website_domain"] 

    if pd.isna(company_name):
        company_name = ""

    if pd.isna(website_domain):
        website_domain = ""

    text = f"{company_name} {website_domain}"
    count = 0
    
    for kw in keywords:
        count += text.count(kw)

    return count
                
df_step3["name_count_keyword"] = df_step3.apply(count_keywords, axis=1).copy()
df_step3["name_count_keyword"].value_counts().sort_index()

/var/folders/yl/brxr15jx04s0b2mrpchp1l9r0000gn/T/ipykernel_28956/1441355304.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_by_keyword_existence)


name_count_keyword
0      90
1     199
2     433
3     114
4     151
5      33
6      45
7      16
8      19
9       3
10      7
11      1
12      3
13      2
14      1
16      1
Name: count, dtype: int64

### Further Improvements
It was identified that a large number of rows contain more than one matching keyword. This suggests an interpratation in which two or more keyword matches can be considered a positive signal for the final classification. 

Despite this observation, an important step is to apply the candidate filtering at input company level rather than globally. For each client-provided company, keyword and location-based signals are evaluated relative to the candidate set associated with that specific input.

Stronger signals are used to narrow down candidates only when weaker alternatives exist, while preserving all candidates in cases where no stronger differentiation is possible. This approach prevents the risk of eliminating the only valid match available for a given company.

For those who have more than one keyword match, an aggregate function can be used to identify the maximum number of keyword occurence count per group and retained at this stage, allowing the analysis to focus on the strongest matches and enabling a clearer final recommendation.

In [16]:
# create function for filtering by max keywords
def filter_by_max_keyword(group):
    max_keywords = group["name_count_keyword"].max()
    return group[group["name_count_keyword"] == max_keywords]
    
df_step4 = (
    df_step3.groupby("input_company_name", group_keys=False)
    .apply(filter_by_max_keyword)
)

# see how the rows are reduced after filtering
df_step4.shape

/var/folders/yl/brxr15jx04s0b2mrpchp1l9r0000gn/T/ipykernel_28956/1604137334.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_by_max_keyword)


(746, 17)

As a next refinement step, company descriptions were used as an additional semantic validation signal. 
Keyword presence within the engine-generated company long descriptions was evaluated at input-company level to further differentiate candidates with similar location and name-based signals.

In [17]:
def description_keyword_match(row):
    keywords = input_name_keywords.loc[row.name]

    description = row["long_description"]

    if pd.isna(description) or not description:
        return False

    description = description.lower()

    return any(kw in description for kw in keywords)

df_step4["description_keyword_match"] = df_step4.apply(description_keyword_match, axis=1)

df_step4["description_keyword_match"].value_counts()

description_keyword_match
True     472
False    274
Name: count, dtype: int64

### Description Refinement
If there is at least one candidate with match description in group, the candidate is kept. If there is no candidate, nothing is removed.

In [18]:
# create fuction for filtering by description
def filter_by_description(group):
    if group["description_keyword_match"].any():
        return group[group["description_keyword_match"] == True]
    else:
        return group

df_step5 = (
    df_step4
    .groupby("input_company_name", group_keys=False)
    .apply(filter_by_description)
)
df_step5.shape

/var/folders/yl/brxr15jx04s0b2mrpchp1l9r0000gn/T/ipykernel_28956/1596395960.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_by_description)


(676, 18)

### Industry-Based Refinement

As a final refinement step, industry information was used as an additional validation signal to further differentiate candidates with otherwise similar location and keyword-based matches.

This refinement is applied at input company level. For each client-provided company, candidates with populated industry information are prioritized only when such data is available within the group. In cases where no candidate has industry information populated, no records are excluded in order to avoid eliminating potentially valid matches.

This step is intended to improve prioritization rather than enforce strict filtering, as industry data may be incomplete or unavailable for some companies.

In [22]:
# create fuction for filtering by industry
def filter_by_industry_presence(group):
    # check if there is at least one candidate with industry populated
    has_industry = group["main_industry"].notna().any()

    if has_industry:
        return group[group["main_industry"].notna()]
    else:
        return group

df_final = (
    df_step5
    .groupby("input_company_name", group_keys=False)
    .apply(filter_by_industry_presence)
)

# create a new column with the original input company name
df_final["input_company_name_original"] = df["input_company_name"]
df_final

/var/folders/yl/brxr15jx04s0b2mrpchp1l9r0000gn/T/ipykernel_28956/440892604.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_by_industry_presence)


,input_company_name,input_main_country,input_main_region,input_main_city,company_name,main_country,main_region,main_city,long_description,main_industry,website_domain,website_url,country_match,region_match,city_match,name_keyword_match,name_count_keyword,description_keyword_match,input_company_name_original
0,24-seven media network (private) limited,pakistan,sindh,karachi,new millennium network,pakistan,sindh,karachi,"incorporated in 2001, mnet is the first compan...",telecommunication services,mnet.com.pk,https://www.mnet.com.pk/,True,True,True,True,1,True,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED
8,2operate a/s,denmark,north denmark region,aalborg,2a pharma aps.,denmark,north denmark region,aalborg,previous next 2a pharma is a clinical stage sw...,pharmaceuticals,2apharma.com,https://2apharma.com/,True,True,True,True,8,True,2OPERATE A/S
9,2operate a/s,denmark,north denmark region,aalborg,rensams solutions,denmark,north denmark region,aalborg,"t rensams solutions ivs,we sell and purchase n...",medical equipment & supplies,rensams.dk,https://rensams.dk/,True,True,True,True,8,True,2OPERATE A/S
560,3 step it a/s,denmark,region of southern denmark,tønder municipality,3 step it a/s.,denmark,capital region of denmark,gladsaxe municipality,NaN,banks & financial services,3stepit.dk,http://www.3stepit.dk/?opi=79508299&sa=u&ved=0...,True,False,False,True,10,False,3 STEP IT A/S
1115,3 step it as,norway,oslo,oslo,nifu,norway,oslo,oslo,the international study of city youth (iscy) e...,research & development services,iscy.no,https://iscy.no/,True,True,True,False,0,True,3 STEP IT AS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706,zte (thailand) co ltd,thailand,bangkok,sathon district,zte thailand,thailand,bangkok,sathon,zte corporation is a global leader in telecomm...,telecommunication services,NaN,NaN,True,True,False,True,2,True,Zte (Thailand) Co Ltd
2712,zte corporation,china,guangdong province,shenzhen,zte,china,guangdong province,shenzhen,zte corporation is a chinese partially state-o...,telecommunication services,zte.com.cn,https://www.zte.com.cn/global/,True,True,True,True,2,True,Zte Corporation
2727,zte norway as,norway,viken,bærum,polypure,norway,viken,bærum,polypure as is a world leader in advanced poly...,chemical products,polypure.com,https://polypure.com/,True,True,True,False,0,True,ZTE NORWAY AS
2729,zte norway as,norway,viken,bærum,bwell norge,norway,viken,bærum,bwell jobber for å redusere stress og tretthet...,therapy & alternative medicine,bwell.no,http://www.bwell.no/,True,True,True,False,0,True,ZTE NORWAY AS


At this stage, the remaining candidate set represents high-confidence matches that are difficult to further differentiate using automated signals alone.

To improve clarity and transparency for the client, an additional column containing the original client-provided company name was created. This allows the resolved entities to be easily traced back to the companies initially provided by the client. 

The final output consists of 660 validated candidates. One representative company from this set is presented below as an illustrative example of a successful resolution.

In [23]:
print(f"Unique candidates with high-confidence match: {df_final['input_company_name'].nunique()}")

Unique candidates with high-confidence match: 563


In [24]:
df_final.shape

(660, 19)

The final dataset consists in 660 valid candidates for 563 client companies.

### Export The Result in CSV Format

In [25]:
df_final.to_csv("veridion_poc_companies.csv", index=False)

### Final Output Overview

After applying the entity resolution and refinement steps, the final dataset contains 563 unique client-provided companies with high-confidence candidate matches. The main features used for matching are location attributes (country, region, and city), keyword matches between the client-provided company name and the engine-provided company name and website domain, the long company description, and the main business category.

While keywords existence signals were used throught the analysis to prioritize and refine candidates where this action was possible, some companies didn't exhibit keyword matches but were intentionally retained to avoid excluding potentially relevant entities in cases where data availability was limited or incomplete.

The resulting dataset represents a clean and reliable shortlist that can be directly used for procurement analysis and decision-making. The full output has been exported as a CSV file to demonstrate readiness and understanding the data.

Below, one representative example from the final high-confidence set is presented to illustrate a clear and well-supported entity resolution outcome.

In [26]:
example_company = df_final[
    df_final["input_company_name"].str.contains("comba", case=False)
]

example_company

,input_company_name,input_main_country,input_main_region,input_main_city,company_name,main_country,main_region,main_city,long_description,main_industry,website_domain,website_url,country_match,region_match,city_match,name_keyword_match,name_count_keyword,description_keyword_match,input_company_name_original
2948,comba technologies sdn. bhd.,malaysia,selangor,subang jaya,cm boost studio,malaysia,selangor,subang jaya,"at cm boost studio, we thrive at the heart of ...",marketing & advertising agencies,cmbooststudio.com,https://www.cmbooststudio.com/,True,True,True,False,0,False,Comba Technologies Sdn. Bhd.
18,comba telecom limited,hong kong,hong kong,hong kong island,comba telecom,hong kong,hong kong,new territories,"today in our digital age, education institutio...",telecommunication services,comba-telecom.com,https://comba-telecom.com/en/,True,True,False,True,4,False,COMBA TELECOM LIMITED
